# Capstone Project - **Concierge Agent** Category 

## *CoMailAgent* – Automated Resume Tailoring & Cold Email Job Outreach Agent 
- by Supriya & Sanya

Find below the code & instructions for this project

### Step 1: Configuring Google API Keys
This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/), which requires an API key.

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

### Step 2: Import ADK components

Now, importing the specific components we will need from the Agent Development Kit and the Generative AI library. This keeps the code organized and ensures we have access to the necessary building blocks.

In [ ]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

### Step 3: Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

1. root_agent - Uses the 
1. get_task_items() tool
    * Goes through the db and returns a dictionary with 
2. cover_letter_agent - for writing a custom cover letter
    * **Extracts keywords and role expectations** from the job description.
    * ~~**Tailors the resume** to highlight relevant skills and achievements.~~
    * **Generates a professional, customized cover letter** for each job.
3. email_agent - for drafting a recruiter outreach email
    * **Creates a personalized cold outreach email** addressed to the recruiter or hiring manager.
    * **Packages attachments** (tailored resume + cover letter).
    * **Sends the email automatically**, or optionally requests user approval before sending.

In [ ]:
def get_job_data(gsheet_id:str) -> dict[list]:
    """Downloads the data from google sheets which contains information about the jobs and recruiter.

        Args:
            gsheet_id: The id of the google sheet

        Returns:
            Dictionary with status and job_data dictionary.
            Success: {"status": "success", "job_data": {'job_id': [1, 2, 3], 
                'job_description':["Description 1", "Description 2", "Description 3"], 
                'recruiter_name': ['racheal x', 'ross y', 'joey z'], 
                'recruiter_email': ['rachael@retail.com', 'ross@museum.com', 'joey@restaurant.com']}}
            Error: {"status": "error", "error_message": "Could not get job data from google sheets"}
        """
    google_sheet_url = f""    

    try:
        # get data from google sheets
        if result:
            job_dataset = {'job_id': [1, 2, 3], 
                        'job_description':["Description 1", "Description 2", "Description 3"], 
                        'recruiter_name': ['racheal x', 'ross y', 'joey z'], 
                        'recruiter_email': ['rachael@retail.com', 'ross@museum.com', 'joey@restaurant.com']}
            return {'Status': 'success', 'job_data':job_dataset}
        else:
            return {'Status': 'error', 'job_data':f"Could not retrive job data as no data available in the db"}
    except Exception as e:
        return {'Status': 'error', 'job_data':f"Could not retrive job data due to the following error: {e}"}


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session